In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout
import dateparser

In [ ]:
def get_content(url):
    page = requests.get(url)
    return page.content

In [ ]:
ttnews_content = get_content('https://www.ttnews.com/articles')

In [ ]:
fw_news_content = get_content('https://www.freightwaves.com/news/category/trucking')

In [94]:
overdrive_business_content = get_content('https://www.overdriveonline.com/business')

In [106]:
samsara_news_content = get_content('https://www.samsara.com/company/news')

In [131]:
platform_science_news_content = get_content('https://www.platformscience.com/in-the-news')

In [ ]:
def get_ttnews(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='views-row')
    
    for article in articles:
        title = article.find('a')
        date = article.find(class_='date-display-single')
        text =  article.find('p') # description
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date['content'])
            companies.append('TTNews')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://ttnews.com' + title.get('href'))
            titles.append(title.getText().strip())
            images.append(image.get('src'))
            company_urls.append('https://ttnews.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

ttnews = get_ttnews(ttnews_content)

ttnews

In [ ]:
def get_overdrive_business(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='node-list__node')
    # print(articles)
    for article in articles:
        title = article.find('h5', class_='section-feed-content-node__content-short-name').a
        # print(title)
        date = article.find(class_='section-feed-content-node__content-published')
        # print(date)
        text =  article.find(class_='section-feed-content-node__content-teaser').a
        # print(text)
        image = article.find('img')

        if (date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Overdrive')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append('https://www.overdriveonline.com' + title.get('href'))
            titles.append(title.getText().strip())
            images.append(image['src'])
            company_urls.append('https://www.overdriveonline.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

overdrive_business = get_overdrive_business(overdrive_business_content)

overdrive_business

In [ ]:
def get_samsara_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='row')
    # print(articles)
    for article in articles:
        title = article.find(class_='source-quote')
        link = article.find('a')
        # print(title)
        date = article.find(class_='source-date')
        # print(date)
        text =  article.find(class_='source-quote')
        # print(text)
        image = article.find('img')
        # print(image)

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Samsara')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(link.get('href'))
            titles.append(title.getText().strip())
            
            if (image != None and image.get('src')):
                images.append(image['src'])
            elif (image != None and image.get('data-lazy')):
                images.append(image['data-lazy'])
            else:
                images.append(None)
            
            company_urls.append('https://www.samsara.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

samsara_news = get_samsara_news(samsara_news_content)

samsara_news

In [141]:
def get_platform_science_news(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []
    
    articles = soup.find_all(class_='media-item')
    # print(articles)
    for article in articles:
        title = article.find(class_='media-heading')
        link = article.find('a')
        # print(title)
        date = article.find(class_='media-date-text')
        # print(date)
        text =  article.find(class_='media-heading')
        # print(text)
        image = article.find(class_='media-image-wrapper')
        # print(image)

        if (link != None and date != None and title != None and text != None):
            parsed_date = dateparser.parse(date.get_text())
            companies.append('Platform Science')
            description.append(text.get_text().strip())
            publish_date.append(parsed_date.date())
            stories.append(link.get('href'))
            titles.append(title.getText().strip())
            
            # <div style="background-image:url(&quot;https://global-uploads.webflow.com/606b95eef43b7bb577930a6d/6148b7d0d8590b7777d1d116_PS_Stevens.jpeg&quot;)" class="media-image-wrapper"></div>
            if (image != None and image.get('style')):
                images.append(image['style'].replace('background-image:url("', '').replace('")', ''))
            else:
                images.append(None)
            
            company_urls.append('https://www.platformscience.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe

platform_science_news = get_platform_science_news(platform_science_news_content)

platform_science_news

,company,headline,description,publish_date,story_url,company_url,image
0,Platform Science,Stevens partners with Platform Science,Stevens partners with Platform Science,2021-09-17,https://www.refrigeratedtransporter.com/carrie...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
1,Platform Science,Automated 'e-Inspections' demonstrated at CVSA...,Automated 'e-Inspections' demonstrated at CVSA...,2021-09-02,https://www.overdriveonline.com/business/artic...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
2,Platform Science,Platform Science partners with E-SMART to impr...,Platform Science partners with E-SMART to impr...,2021-08-26,https://ajot.com/news/article/platform-science...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
3,Platform Science,Three states automating Level 3 inspections at...,Three states automating Level 3 inspections at...,2021-07-05,https://www.ccjdigital.com/technology/article/...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
4,Platform Science,Drivewyze Opens Enrollment for e-Inspections a...,Drivewyze Opens Enrollment for e-Inspections a...,2021-06-29,https://www.truckinginfo.com/10146487/drivewyz...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
...,...,...,...,...,...,...,...
91,Platform Science,FreightTech 25 and Platform Science Consolidat...,FreightTech 25 and Platform Science Consolidat...,2018-11-14,https://bit.ly/2FUEeLz,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
92,Platform Science,MarketWaves18’s Demo Day Best in Show Announced,MarketWaves18’s Demo Day Best in Show Announced,2018-11-14,https://bit.ly/2S9m8LD,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
93,Platform Science,Platform Science Integrates with Drivewyze Byp...,Platform Science Integrates with Drivewyze Byp...,2018-08-16,https://bit.ly/2FXwdpc,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
94,Platform Science,"Schneider, 8VC Partner to Boost Digital Supply...","Schneider, 8VC Partner to Boost Digital Supply...",2018-04-11,https://www.ttnews.com/articles/schneider-8vc-...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...


In [ ]:
def get_freightWaves(content):
    soup = BeautifulSoup(content, "html.parser")

    companies = []
    description = []
    publish_date = []
    company_urls = []
    stories = []
    titles = []
    images = []

    articles = soup.find_all(class_='post-item')
    
    for article in articles:
        title = article.find('a')
        date =  article.find(class_='date meta-item tie-icon')
        text =  article.find('p')
        image = article.find('img')

        companies.append('FreightWaves')
        description.append(text.getText())
        publish_date.append(dateparser.parse(date.get_text()).date())
        stories.append(title.get('href'))
        titles.append(title.get('aria-label'))
        images.append(image)
        company_urls.append('https://www.freightwaves.com/')

    dataframe = pd.DataFrame({'company':companies, 'headline':titles, 'description':description, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls, 'image':images})
    return dataframe
        
fw_news = get_freightWaves(fw_news_content)

fw_news

In [142]:
final = pd.concat([ttnews, fw_news, overdrive_business, samsara_news, platform_science_news], axis = 0)
#replace all .coms in company names
final['company'] = final.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

In [143]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,https://ttnews.com/articles/atri-report-urges-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,https://ttnews.com/articles/transportation-sta...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,https://ttnews.com/articles/senate-passes-fund...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
...,...,...,...,...,...,...,...
91,Platform Science,FreightTech 25 and Platform Science Consolidat...,FreightTech 25 and Platform Science Consolidat...,2018-11-14,https://bit.ly/2FUEeLz,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
92,Platform Science,MarketWaves18’s Demo Day Best in Show Announced,MarketWaves18’s Demo Day Best in Show Announced,2018-11-14,https://bit.ly/2S9m8LD,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
93,Platform Science,Platform Science Integrates with Drivewyze Byp...,Platform Science Integrates with Drivewyze Byp...,2018-08-16,https://bit.ly/2FXwdpc,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
94,Platform Science,"Schneider, 8VC Partner to Boost Digital Supply...","Schneider, 8VC Partner to Boost Digital Supply...",2018-04-11,https://www.ttnews.com/articles/schneider-8vc-...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...


In [ ]:
final.head()

In [ ]:
len(final)

In [ ]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

In [ ]:
len(final)

In [144]:
final

,company,headline,description,publish_date,story_url,company_url,image
0,TTNews,"No Vote Held on Infrastructure, but Talks to R...",WASHINGTON — Despite a long night of frantic n...,2021-10-01,https://ttnews.com/articles/no-vote-held-infra...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
1,TTNews,TT Newsmakers,"Sgt. Zach Heard, Illinois State Police, and Ke...",2021-09-30,https://ttnews.com/newsmakers,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
2,TTNews,ATRI Report Urges Electric Tax to Help Fund Hi...,A new report from the American Transportation ...,2021-09-30,https://ttnews.com/articles/atri-report-urges-...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
3,TTNews,Transportation Stakeholders Press Congress on ...,To guarantee long-term funding for the country...,2021-09-30,https://ttnews.com/articles/transportation-sta...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
4,TTNews,Congress Clears Government Funding Bill to Ave...,The House passed a nine-week spending bill to ...,2021-09-30,https://ttnews.com/articles/senate-passes-fund...,https://ttnews.com/,https://h7f7z2r7.stackpathcdn.com/sites/defaul...
...,...,...,...,...,...,...,...
91,Platform Science,FreightTech 25 and Platform Science Consolidat...,FreightTech 25 and Platform Science Consolidat...,2018-11-14,https://bit.ly/2FUEeLz,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
92,Platform Science,MarketWaves18’s Demo Day Best in Show Announced,MarketWaves18’s Demo Day Best in Show Announced,2018-11-14,https://bit.ly/2S9m8LD,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
93,Platform Science,Platform Science Integrates with Drivewyze Byp...,Platform Science Integrates with Drivewyze Byp...,2018-08-16,https://bit.ly/2FXwdpc,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...
94,Platform Science,"Schneider, 8VC Partner to Boost Digital Supply...","Schneider, 8VC Partner to Boost Digital Supply...",2018-04-11,https://www.ttnews.com/articles/schneider-8vc-...,https://www.platformscience.com/,https://global-uploads.webflow.com/606b95eef43...


In [145]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False)

In [105]:
len(new_old)

48

In [ ]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [ ]:
print(title)
pyperclip.copy(title)

In [ ]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
final_dict = final_dict[0:7]
for row in final_dict:
    headline = row['headline']
    company_name = row['company']
    publish_date = row['publish_date']
    description = row['description']
    story_url = row['story_url']
    company_url = row['company_url']
    img = row['image']

    row_url = company_url + story_url
    
    entry = '''
    <tr>
      <td>
        {img}
        <h3>
          <a href = "{row_url}">{headline}</a>
        </h3>
        <p>
          {description}
        </p>
        <p>
          <a href = "{company_url}">{company_name}</a>
        </p>
        <p>
          <b>{publish_date}</b>
        </p>
      </td>
    </tr> '''.format(row_url = row_url, headline = headline, company_url = company_url, company_name = company_name, description = description, publish_date = publish_date, img=img).strip()

    content += '\n' + entry.strip()

In [130]:
pyperclip.copy(content)
print(content)

PythonQtError: No Qt bindings could be found

In [ ]:
#job boards
# stackoverflow, jobvite, bamboohr